In [2]:

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

import os
import glob
import copy

from pprint import pprint
import itertools as it
from collections import Counter
import numpy as np

from aiida import load_profile
from aiida.orm.nodes.data.structure import StructureData

from aiida_quantumespresso.tools.pwinputparser import create_builder_from_file
from aiida_quantumespresso.calculations.pw import PwCalculation
from aiida_quantumespresso.calculations.pwimmigrant import PwimmigrantCalculation
# from aiida_quantumespresso.tools import pwinputparser

from pymatgen.io.ase import AseAtomsAdaptor

# import pybat
# from pybat import Cathode, LiRichCathode, Dimer, DimerNEBAnalysis

from ase.visualize import view
from ase.io.vasp import read_vasp, write_vasp
from ase.build.tools import sort

In [3]:
load_profile()

Profile<uuid='4fe00b7a32994cfca8b6cf43c2d22a53' name='jgeiger'>

### Read in fully lithiated phospho-olivine structure from pw calculation

In [4]:
iurii_olivine_dir = '/home/jgeiger/projects/bat_uv_ml/data/olivines_iurii/LixMnPO4/Li1.00/DFT_plus_U/1_vcrelax/'
iurii_olivine_file = 'LiMnPO4.vcrelax.1.in'
iurii_pseudo_dir = '/home/jgeiger/projects/bat_uv_ml/data/olivines_iurii/Pseudopotentials/SSSP'

disorder_dir = os.path.join(iurii_olivine_dir, 'disorder_sampling')

In [5]:
builder = create_builder_from_file(
    input_folder=iurii_olivine_dir,
    input_file_name=iurii_olivine_file,
    code='qe-6.4.1-pw',
    metadata={},
    pseudo_folder_path=iurii_pseudo_dir,
)

structuredata = builder['structure']

initial_ase = structuredata.get_ase()
initial_ase = sort(initial_ase)

initial_pmg = AseAtomsAdaptor.get_structure(initial_ase)

In [6]:
for repeat in list(it.product((1,2), repeat=3)):
    
    ase_structure = copy.deepcopy(initial_ase)
    ase_structure *= repeat
    ase_structure = sort(ase_structure)
    
    li_number = ase_structure.get_chemical_symbols().count('Li')
    repeat_dir = os.path.join(
        disorder_dir,
        'Li{:02d}_{}'.format(
            li_number,
            'x'.join(map(str, repeat))
            )
    )
    
    # os.makedirs(repeat_dir, exist_ok=True)
    # write_vasp(file=os.path.join(repeat_dir, 'POSCAR'), atoms=ase_structure)

***
### Read in poscars into ase.Atoms movie for visualization

In [7]:
# Python find all directories with name pos_

# print(disorder_dir)

poscar_paths = []

for root, dirs, files in os.walk(os.path.join(disorder_dir, 'Li16_1x2x2')):
    for dir in dirs:
        if 'pos_' in dir:
            poscar_paths.append(os.path.join(root, dir, 'POSCAR'))

poscar_ase_list = [read_vasp(poscar_path) for poscar_path in poscar_paths]
# view(poscar_ase_list)

***
### Immigrate PW calculation from Iurii

In [21]:
# test_pw_dir = '/home/jgeiger/devel/aiida-quantumespresso/tests/calculations/test_pw/test_pw_default.in'
# pw_immigrant = PwimmigrantCalculation() # ! Deprecated

***
### Try out sampling with `icet`

In [ ]:
from pymatgen.core import Structure
# from Cathode import from_structure

# pmg_structure = Structure()
# pprint(pmg_structure.__dict__, sort_dicts=False)
initial_cat = Cathode.from_structure(initial_pmg)
# print(initial_pmg.__dict__)
print(initial_cat)

Full Formula (Li4 Mn4 P4 O16)
Reduced Formula: LiMnPO4
abc   :  10.363227   6.057852   4.719737
angles:  90.000000  89.999994  90.000000
Sites (28)
  #    #VESTA  SP            a          b          c    masses    tags
---  --------  ----  ---------  ---------  ---------  --------  ------
  0         1  Li    -0.282154  -0.249998  -0.972215         5       0
  1         2  Li     0.217845  -0.250001  -0.472214         5       0
  2         3  Li     0.217845   0.250001  -0.472214         5       0
  3         4  Li    -0.282154   0.249998  -0.972215         5       0
  4         5  Mn    -0.001476   0          0.004523        56       1
  5         6  Mn     0.498524   0         -0.44893         56       1
  6         7  Mn    -0.062831   0.5       -0.495478        56       2
  7         8  Mn     0.437169   0.5       -0.948929        56       2
  8         9  O     -0.187352   0         -0.232027        16       1
  9        10  O      0.623041   0.5       -0.712391        16       1


- Results for small cell the same, using either `disorder` or `icet`, but have to use symmetry threshold of 10-6, with 10-4 the former finds less configurations
- 

In [20]:
import importlib

import pybat.core
# ! Importlib does not work when shorthand notation without core is skipped. Need to be explicit.
importlib.reload(pybat.core)

initial_cat = pybat.core.Cathode.from_structure(initial_pmg)

cathode_list = initial_cat.get_cation_configurations(
    substitution_sites=range(0,4),
    cation_list=["Li", "Vac"],
    sizes=[1], # , 2],
    # ! -> Up to twice the atom count, but also taking into account other symmetries
    concentration_restrictions={"Li": (0, 1)},
    max_configurations=None,
)

pmg_list = [cathode.as_ordered_structure() for cathode in cathode_list]
ase_list = [AseAtomsAdaptor.get_atoms(pmg_) for pmg_ in pmg_list]

# view(ase_list)

<module 'pybat.core' from '/home/jgeiger/devel/pybat/pybat/core.py'>

[['Li', 'Lr'], ['Li', 'Lr'], ['Li', 'Lr'], ['Li', 'Lr'], ['Mn'], ['Mn'], ['Mn'], ['Mn'], ['O'], ['O'], ['O'], ['O'], ['O'], ['O'], ['O'], ['O'], ['O'], ['O'], ['O'], ['O'], ['O'], ['O'], ['O'], ['O'], ['P'], ['P'], ['P'], ['P']]
test
No magnetic moments found in structure, setting to zero.


In [59]:
li_number_list = [_.get_chemical_symbols().count('Li') for _ in ase_list]
pprint(li_number_list)

view(ase_list)

[4, 3, 3, 2, 2, 2, 2, 1, 1, 0]


<Popen: returncode: None args: ['/home/jgeiger/anaconda3/envs/aiida/bin/pyth...>

In [55]:

mn_number_list = [_.get_chemical_symbols().count('Mn') for _ in ase_list]
mn_number_string = ''.join(map(str, mn_number_list))

cell_list = [repr(_.cell) for _ in ase_list]
pprint(cell_list[:2])

Counter('_'.join(cell_list[2]))

# type(ase_list[0].cell)
# from ase.cell import Cell

['Cell([[2.5087278084897446e-06, 0.0, -4.719736618435828], [0.0, '
 '-6.0578517679515285, 0.0], [-10.363227234350049, 0.0, '
 '-6.545405064125506e-06]])',
 'Cell([[2.5087278084897446e-06, 0.0, -4.719736618435828], [0.0, '
 '-6.0578517679515285, 0.0], [-10.363227234350049, 0.0, '
 '-6.545405064125506e-06]])']


Counter({'C': 1,
         '_': 141,
         'e': 3,
         'l': 2,
         '(': 1,
         '[': 4,
         '2': 8,
         '.': 9,
         '5': 13,
         '0': 19,
         '8': 9,
         '7': 9,
         '4': 10,
         '9': 4,
         '6': 11,
         '-': 6,
         ',': 8,
         ' ': 8,
         '1': 6,
         '3': 6,
         ']': 4,
         ')': 1})

***
### Bash cell for verdi stuff

In [ ]:
%%bash
# verdi storage info
verdi data --help